Step 1. Install data from google drive

In [ ]:
! rm -rf /content/datasets
! mkdir /content/datasets

In [ ]:
!pip install --upgrade --no-cache-dir gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! pwd

/content


In [ ]:
# Dowmload picture and annotation file
!gdown https://drive.google.com/uc?id=

Downloading...
From: https://drive.google.com/uc?id=1gNLAU3h2WyoNXsfH_6nO0NR5u6xfLjpX
To: /content/PWMFD_All_9000.zip
100% 1.34G/1.34G [00:11<00:00, 121MB/s]


In [ ]:
!unzip -q PWMFD_All_9000.zip -d /content/datasets && rm PWMFD_All_9000.zip

In [ ]:
# Dowmload .yaml file
!gdown https://drive.google.com/uc?id=

Downloading...
From: https://drive.google.com/uc?id=1OloxRd2erkiAC4fCKMQHxVEEu-XwwxaB
To: /content/data_9000.yaml
100% 862/862 [00:00<00:00, 1.41MB/s]


In [ ]:
!mv /content/data_9000.yaml /content/datasets/PWMFD_All_9000/

Step 2. Data preprocessing: 

transform xml to .txt 

In [ ]:
# Define Data ETL function 
from bs4 import BeautifulSoup
import os
import shutil

status_dic = {'with_mask': 2, 'incorrect_mask': 1, 'without_mask': 0}
def getYoloFormat(filename,label_path, img_path, yolo_path, newname):
    with open(label_path+ filename, 'r') as f:
        soup = BeautifulSoup(f.read(), 'xml')
        imgname = soup.select_one('filename').text
        image_w = int(soup.select_one('width').text)
        image_h = int(soup.select_one('height').text)
        ary = []
        for obj in soup.select('object'):
            xmin = int(obj.select_one('xmin').text)
            xmax = int(obj.select_one('xmax').text)
            ymin = int(obj.select_one('ymin').text)
            ymax = int(obj.select_one('ymax').text)
            objclass = status_dic.get(obj.select_one('name').text)

            x = (xmin + (xmax-xmin)/2) * 1.0 / image_w
            y = (ymin + (ymax-ymin)/2) * 1.0 / image_h
            w = (xmax-xmin) * 1.0 / image_w
            h = (ymax-ymin) * 1.0 / image_h
            ary.append(' '.join([str(objclass), str(x),str(y),str(w),str(h)]))
        if os.path.exists(img_path + imgname):
            #shutil.copyfile(img_path + imgname, yolo_path + newname + '.jpg')
            with open(yolo_path + filename[0:6] + '.txt', 'w') as f:
                f.write('\n'.join(ary))

In [ ]:
# transform train data
import os
labelpath = '/content/datasets/PWMFD_All_9000/Annotations/train/'
imgpath   = '/content/datasets/PWMFD_All_9000/images/train/'
yolopath  = '/content/datasets/PWMFD_All_9000/labels/train/'
ary = []
for idx, f in enumerate(os.listdir(labelpath)):
    try:
        getYoloFormat(f, labelpath,imgpath, yolopath, str(idx))
    except Exception as e:
        print(e)

'NoneType' object has no attribute 'text'


In [ ]:
# transform validation data
import os
labelpath = '/content/datasets/PWMFD_All_9000/Annotations/val/'
imgpath   = '/content/datasets/PWMFD_All_9000/images/val/'
yolopath  = '/content/datasets/PWMFD_All_9000/labels/val/'
ary = []
for idx, f in enumerate(os.listdir(labelpath)):
    try:
        getYoloFormat(f, labelpath,imgpath, yolopath, str(idx))
    except Exception as e:
        print(e)

'NoneType' object has no attribute 'text'


In [ ]:
# view the data
! cat /content/datasets/PWMFD_All_9000/labels/train/000002.txt

2 0.5728542914171657 0.5147058823529411 0.4151696606786427 0.713903743315508

Step 3. train our model

In [ ]:
# git clone yolov5 first
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
from yolov5 import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v6.1-223-g1dcb774 Python-3.7.13 torch-1.11.0+cu113 CPU


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 39.7/107.7 GB disk)


In [ ]:
! pwd

/content/yolov5


In [ ]:
# Train our data by yolov5s
!python train.py --img 320 --batch 16 --epochs 50 --data /content/datasets/PWMFD_All_9000/data_9000.yaml --weights yolov5s.pt --cache

train: weights=yolov5s.pt, cfg=, data=/content/datasets/PWMFD_All_9000/data_9000.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=320, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-223-g1dcb774 Python-3.7.13 torch-1.11.0+cu113 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.

Step 4. save our model

In [ ]:
# mount the google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Save if the data is not null!

In [ ]:
! zip -r exp.zip /content/yolov5/runs/train/exp/

  adding: content/yolov5/runs/train/exp/ (stored 0%)
  adding: content/yolov5/runs/train/exp/opt.yaml (deflated 40%)
  adding: content/yolov5/runs/train/exp/weights/ (stored 0%)
  adding: content/yolov5/runs/train/exp/labels.jpg (deflated 20%)
  adding: content/yolov5/runs/train/exp/train_batch0.jpg (deflated 3%)
  adding: content/yolov5/runs/train/exp/train_batch1.jpg (deflated 3%)
  adding: content/yolov5/runs/train/exp/events.out.tfevents.1646187465.6f86d262de38.402.0 (deflated 93%)
  adding: content/yolov5/runs/train/exp/labels_correlogram.jpg (deflated 19%)
  adding: content/yolov5/runs/train/exp/train_batch2.jpg (deflated 2%)
  adding: content/yolov5/runs/train/exp/hyp.yaml (deflated 45%)


In [ ]:
! pwd

/content/yolov5


In [ ]:
! mv /content/yolov5/exp.zip /content/drive/MyDrive/